IMPORTING LIBRARY

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

EDA

In [2]:
main_df = pd.read_csv('/kaggle/input/tmdb-movies-dataset-2023-930k-movies/TMDB_movie_dataset_v11.csv')

In [3]:
df = main_df[main_df['vote_average']!=0]

In [4]:
df.reset_index(inplace=True)

In [5]:
df.shape

(351611, 25)

In [6]:
df.head()

,index,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


Features Selection

In [7]:
df.columns

Index(['index', 'id', 'title', 'vote_average', 'vote_count', 'status',
       'release_date', 'revenue', 'runtime', 'adult', 'backdrop_path',
       'budget', 'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [8]:
df = df.drop( ['index','id' , 'vote_count' , 'status' , 'release_date', 'revenue' , 'backdrop_path',
              'budget','homepage','imdb_id','original_title' , 'overview','poster_path',
              'tagline' , 'production_companies','production_countries' ,'spoken_languages' ,'keywords'], axis=1)

In [9]:
df.head()

,title,vote_average,runtime,adult,original_language,popularity,genres
0,Inception,8.364,148,False,en,83.952,"Action, Science Fiction, Adventure"
1,Interstellar,8.417,169,False,en,140.241,"Adventure, Drama, Science Fiction"
2,The Dark Knight,8.512,152,False,en,130.643,"Drama, Action, Crime, Thriller"
3,Avatar,7.573,162,False,en,79.932,"Action, Adventure, Fantasy, Science Fiction"
4,The Avengers,7.710,143,False,en,98.082,"Science Fiction, Action, Adventure"


In [10]:
df['org_title']=df['title']

In [11]:
df.isna().sum()

title                    0
vote_average             0
runtime                  0
adult                    0
original_language        0
popularity               0
genres               59690
org_title                0
dtype: int64

In [12]:
df['genres'] = df['genres'].fillna('unknown')

In [13]:
df.isna().sum()

title                0
vote_average         0
runtime              0
adult                0
original_language    0
popularity           0
genres               0
org_title            0
dtype: int64

In [14]:
df.duplicated().sum()

39

In [15]:
df = df.drop_duplicates()

In [16]:
df.duplicated().sum()

0

In [17]:
dff= df.copy()

MultiLabel Encoder¶

In [18]:
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

genre_l = dff['genres'].apply(lambda x: x.split(','))
genre_l = pd.DataFrame(genre_l)

In [19]:
genre_l

,genres
0,"[Action, Science Fiction, Adventure]"
1,"[Adventure, Drama, Science Fiction]"
2,"[Drama, Action, Crime, Thriller]"
3,"[Action, Adventure, Fantasy, Science Fiction]"
4,"[Science Fiction, Action, Adventure]"
...,...
351606,[unknown]
351607,"[Science Fiction, Drama]"
351608,"[Music, Documentary]"
351609,[Documentary]


In [20]:
genre_l['genres'] = genre_l['genres'].apply(lambda x :[ y.strip().lower().replace(' ','') for y in x] )

In [21]:
MLB = MultiLabelBinarizer()

genre_encoded = MLB.fit_transform(genre_l['genres'])

In [22]:
genre_encoded

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [23]:
genre_encoded_df = pd.DataFrame(genre_encoded, columns=MLB.classes_)
genre_encoded_df=genre_encoded_df.reset_index()

In [24]:
mod_df = dff.drop(['genres'],axis=1)
mod_df=mod_df.reset_index()

In [25]:
df = pd.concat([mod_df,genre_encoded_df],axis=1).drop('index',axis=1)

In [26]:
df.head()

,title,vote_average,runtime,adult,original_language,popularity,org_title,action,adventure,animation,...,horror,music,mystery,romance,sciencefiction,thriller,tvmovie,unknown,war,western
0,Inception,8.364,148,False,en,83.952,Inception,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,Interstellar,8.417,169,False,en,140.241,Interstellar,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,The Dark Knight,8.512,152,False,en,130.643,The Dark Knight,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Avatar,7.573,162,False,en,79.932,Avatar,1,1,0,...,0,0,0,0,1,0,0,0,0,0
4,The Avengers,7.710,143,False,en,98.082,The Avengers,1,1,0,...,0,0,0,0,1,0,0,0,0,0


Features Engineering

In [27]:
df['title'] = df['title'].apply(lambda x :x.strip().lower().replace(' ','') )
df['original_language'] = df['original_language'].apply(lambda x :x.strip().lower().replace(' ','') )


In [28]:
df.head()

,title,vote_average,runtime,adult,original_language,popularity,org_title,action,adventure,animation,...,horror,music,mystery,romance,sciencefiction,thriller,tvmovie,unknown,war,western
0,inception,8.364,148,False,en,83.952,Inception,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,interstellar,8.417,169,False,en,140.241,Interstellar,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,thedarkknight,8.512,152,False,en,130.643,The Dark Knight,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,avatar,7.573,162,False,en,79.932,Avatar,1,1,0,...,0,0,0,0,1,0,0,0,0,0
4,theavengers,7.710,143,False,en,98.082,The Avengers,1,1,0,...,0,0,0,0,1,0,0,0,0,0


In [29]:
df.loc[~( (df['original_language']=='en')|(df['original_language']=='fr')|(df['original_language']=='es')|(df['original_language']=='de')|(df['original_language']=='ja')),'original_language'] = 'else'

One-Hot Encoding

In [30]:
OHE = OneHotEncoder(sparse_output=False)

In [31]:
df['adult'] = df['adult'].astype('str')
adult_enc = OHE.fit_transform(df[['adult']])
adult_enc_df = pd.DataFrame(adult_enc,columns=OHE.get_feature_names_out())


In [32]:
adult_enc_df = adult_enc_df.drop('adult_True',axis=1)

In [33]:
lang_enc = OHE.fit_transform(df[['original_language']])
lang_enc_df = pd.DataFrame(lang_enc,columns=OHE.get_feature_names_out())

In [34]:
mod_df = df.drop(['adult','original_language'],axis=1)

In [35]:
df = pd.concat([mod_df,adult_enc_df,lang_enc_df],axis=1)

In [36]:
df.head()

,title,vote_average,runtime,popularity,org_title,action,adventure,animation,comedy,crime,...,unknown,war,western,adult_False,original_language_de,original_language_else,original_language_en,original_language_es,original_language_fr,original_language_ja
0,inception,8.364,148,83.952,Inception,1,1,0,0,0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,interstellar,8.417,169,140.241,Interstellar,0,1,0,0,0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,thedarkknight,8.512,152,130.643,The Dark Knight,1,0,0,0,1,...,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,avatar,7.573,162,79.932,Avatar,1,1,0,0,0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,theavengers,7.710,143,98.082,The Avengers,1,1,0,0,0,...,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


Normalization

In [37]:
from sklearn.preprocessing import StandardScaler
SC = StandardScaler()

In [38]:
df_norm = SC.fit_transform(df.drop(['title','org_title'],axis=1))
df_norm_df = pd.DataFrame(df_norm, columns=[x for x in df.columns if x not in ['title', 'org_title']])

In [39]:
df = pd.concat([df[['title','org_title']],df_norm_df],axis=1)

In [40]:
df.head()

,title,org_title,vote_average,runtime,popularity,action,adventure,animation,comedy,crime,...,unknown,war,western,adult_False,original_language_de,original_language_else,original_language_en,original_language_es,original_language_fr,original_language_ja
0,inception,Inception,1.142404,1.197553,6.077554,3.469316,4.802478,-0.254765,-0.52443,-0.246249,...,-0.452108,-0.13278,-0.113892,0.244856,-0.214454,-0.589881,0.962327,-0.272904,-0.275375,-0.201189
1,interstellar,Interstellar,1.169278,1.520199,10.282470,-0.288241,4.802478,-0.254765,-0.52443,-0.246249,...,-0.452108,-0.13278,-0.113892,0.244856,-0.214454,-0.589881,0.962327,-0.272904,-0.275375,-0.201189
2,thedarkknight,The Dark Knight,1.217448,1.259009,9.565478,3.469316,-0.208226,-0.254765,-0.52443,4.060929,...,-0.452108,-0.13278,-0.113892,0.244856,-0.214454,-0.589881,0.962327,-0.272904,-0.275375,-0.201189
3,avatar,Avatar,0.741320,1.412650,5.777251,3.469316,4.802478,-0.254765,-0.52443,-0.246249,...,-0.452108,-0.13278,-0.113892,0.244856,-0.214454,-0.589881,0.962327,-0.272904,-0.275375,-0.201189
4,theavengers,The Avengers,0.810787,1.120732,7.133097,3.469316,4.802478,-0.254765,-0.52443,-0.246249,...,-0.452108,-0.13278,-0.113892,0.244856,-0.214454,-0.589881,0.962327,-0.272904,-0.275375,-0.201189


Handling Duplicates

In [41]:
df = df.drop_duplicates(subset=['title'])

In [42]:
df=df.set_index(['title'])
df_fin=df.drop('org_title',axis=1)

Cosine-Similatry

In [43]:
movie_name = 'the dark knight'
movie_name=movie_name.strip().lower().replace(' ','')
new_df= df_fin.loc[[movie_name]]

In [44]:
new_df = new_df.values.reshape(1,-1)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity
df_other = df_fin.loc[df_fin.index!=movie_name,:]
df_titles = df.loc[df.index!=movie_name,'org_title']
cosine_sim_matrix = cosine_similarity(new_df,df_other)
cosine_sim_df = pd.DataFrame(cosine_sim_matrix,index=[movie_name],columns=df_titles)

cosine_sim_df

org_title,Inception,Interstellar,Avatar,The Avengers,Deadpool,Avengers: Infinity War,Fight Club,Guardians of the Galaxy,Pulp Fiction,Forrest Gump,...,Me gusta esa chica,Raymonde El Bidaoia,Trona Pinnacles,Human Energy,Jízdní hlídka,Triomf,Kizu: The Untold Story of Unit 731,"Jessico, Una Historia de Rock en Tiempos Convulsos",Henry Kissinger: Secrets of a Superpower,Dastaan
thedarkknight,0.582434,0.669166,0.494289,0.623935,0.620006,0.729754,0.778864,0.334624,0.886324,0.693369,...,-0.153003,-0.138255,-0.075378,-0.174864,-0.048081,-0.085024,-0.087126,-0.08569,-0.05156,-0.157936


In [46]:
sorted_row = cosine_sim_df.loc[movie_name].sort_values(ascending=False)[0:20]

In [47]:
sorted_row.index[5]

'John Wick: Chapter 3 - Parabellum'

Deployment

In [48]:
!pip install gradio
import gradio as gr
from sklearn.metrics.pairwise import cosine_similarity
def predict(movie_name,no_movies):
    movie_name=movie_name.strip().lower().replace(' ','')
    if(movie_name in df_fin.index):
        new_df= df_fin.loc[[movie_name]]
        new_df = new_df.values.reshape(1,-1)
        df_other = df_fin.loc[df_fin.index!=movie_name,:]
        df_titles = df.loc[df.index!=movie_name,'org_title']
        cosine_sim_matrix = cosine_similarity(new_df,df_other)
        cosine_sim_df = pd.DataFrame(cosine_sim_matrix,index=[movie_name],columns=df_titles)
        sorted_row = cosine_sim_df.loc[movie_name].sort_values(ascending=False)[0:int(no_movies)]
        rec=''
        for i in range(int(no_movies)):
            rec+=(sorted_row.index[i])+'\n'
        return rec
    else:
        return "Sorry, this movie isn't in our database. \n try another one !"

# Define the Gradio interface
interface = gr.Interface(
    fn=predict,  # The function to be called
    inputs=[gr.Textbox(label="Movie Name : "),
            gr.Textbox(label='No.of Recommendations: ',value= '5')],# Input type
    outputs=gr.Textbox(label="Recommendations : ")  # Output type
)
interface.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://a9a25366b3b3895bde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from 